# 1. Título - Indicadores sobre Ensino Superior

# 2. Membros
## 2020089879 - Chrystian Melo
## 2022043787 - Fernando Coutinho
## 2022037507 - Murilo Ribeiro
## 2020006914 - Nauan Barbosa

# 3. Descrição dos dados (qual a URL? qual o domínio? como os dados foram processados?)

[Dados do MEC](https://dados.gov.br/dados/conjuntos-dados/indicadores-sobre-ensino-superior) sobre Ensino Superior

## Processamento dos dados
.csv -> .db

# 4. Diagrama ER

(pendente)

# 5. Diagrama relacional

## https://docs.google.com/document/d/1r1sGPKGszgGQEFryLsbwUbDDY_2dZFVkm-qaL7ASRF8/edit?usp=sharing

# 6. Consultas

## Preparação do ambiente

In [1]:
import io
import pandas as pd
import json
import sqlite3
import json

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

graduacao =  pd.read_csv("/content/drive/MyDrive/Colab Notebooks/IBD/tp2/Graduacao.csv")
especializacao = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/IBD/tp2/Especializacao.csv")

conn = sqlite3.connect('mecData.db')

graduacao.to_sql('Graduacao', conn, if_exists='replace', index=False)
especializacao.to_sql('Especializacao', conn, if_exists='replace', index=False)

cursor = conn.cursor()

Drive not mounted, so nothing to flush and unmount.


MessageError: ignored

## Normalização

In [ ]:
query = """
SELECT DISTINCT
  CODIGO_AREA_OCDE_CINE AS CODIGO_AREA_CONHECIMENTO, AREA_OCDE_CINE AS NOME_AREA_CONHECIMENTO
FROM
  graduacao
GROUP BY CODIGO_AREA_CONHECIMENTO
UNION
SELECT DISTINCT
  CODIGO_OCDE_CINE AS CODIGO_AREA_CONHECIMENTO, OCDE_CINE AS NOME_AREA_CONHECIMENTO
FROM
  especializacao
GROUP BY CODIGO_AREA_CONHECIMENTO
"""
df = pd.read_sql_query(query, conn)
df.to_sql('Tematica', conn, if_exists='replace', index=False)

cursor.execute("ALTER TABLE graduacao DROP COLUMN AREA_OCDE_CINE;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN AREA_OCDE;") #coluna com dado duplicado

cursor.execute("ALTER TABLE especializacao DROP COLUMN OCDE_CINE;")

conn.commit()

print (df)

    CODIGO_AREA_CONHECIMENTO                       NOME_AREA_CONHECIMENTO
0                       None                                         None
1                        0.0                            Programas básicos
2                        1.0                                     Educação
3                        2.0                          Artes e humanidades
4                        3.0   Ciências sociais, comunicação e informação
..                       ...                                          ...
431                       95  Conservação e restauro de material cultural
432                       96                                     História
433                       97                                    Filosofia
434                       98    Línguas/literaturas estrangeiras modernas
435                       99         Redação criativa / expressão escrita

[436 rows x 2 columns]


In [ ]:
query = """
SELECT DISTINCT
  CODIGO_MUNICIPIO, MUNICIPIO AS NOME_MUNICIPIO, UF, REGIAO
FROM
  graduacao
UNION
SELECT DISTINCT
  CODIGO_MUNICIPIO, MUNICIPIO AS NOME_MUNICIPIO, UF, REGIAO
FROM
  especializacao
"""

df = pd.read_sql_query(query, conn)
df.to_sql('Municipio', conn, if_exists='replace', index=False)

cursor.execute("ALTER TABLE graduacao DROP COLUMN MUNICIPIO;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN UF;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN REGIAO;")

cursor.execute("ALTER TABLE especializacao DROP COLUMN MUNICIPIO;")
cursor.execute("ALTER TABLE especializacao DROP COLUMN UF;")
cursor.execute("ALTER TABLE especializacao DROP COLUMN REGIAO;")

conn.commit()

print (df)

      CODIGO_MUNICIPIO         NOME_MUNICIPIO  UF             REGIAO
0              1100015  Alta Floresta D´Oeste  RO              NORTE
1              1100023              Ariquemes  RO              NORTE
2              1100049                 Cacoal  RO              NORTE
3              1100056             Cerejeiras  RO              NORTE
4              1100064      Colorado do Oeste  RO              NORTE
...                ...                    ...  ..                ...
3785           5221908                 Varjão  GO       CENTRO-OESTE
3786           5222005             Vianópolis  GO       CENTRO-OESTE
3787           5222054         Vicentinópolis  GO       CENTRO-OESTE
3788           5300108               Brasília  DF       CENTRO-OESTE
3789           9000001   Ignorado ou exterior  ZZ  IGNORADO/EXTERIOR

[3790 rows x 4 columns]


In [ ]:
query = """
SELECT DISTINCT
  CODIGO_IES AS CODIGO_INSTITUICAO, NOME_IES AS NOME_INSTITUICAO
FROM
  graduacao
UNION
SELECT DISTINCT
  CODIGO_IES AS CODIGO_INSTITUICAO, NOME_IES AS NOME_INSTITUICAO
FROM
  especializacao
"""

df = pd.read_sql_query(query, conn)
df.to_sql('Instituicao', conn, if_exists='replace', index=False)

cursor.execute("ALTER TABLE graduacao DROP COLUMN NOME_IES;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN CATEGORIA_ADMINISTRATIVA;")
cursor.execute("ALTER TABLE graduacao DROP COLUMN ORGANIZACAO_ACADEMICA;")

cursor.execute("ALTER TABLE especializacao DROP COLUMN NOME_IES;")

conn.commit()

print (df)

      CODIGO_INSTITUICAO                                   NOME_INSTITUICAO
0                      1                UNIVERSIDADE FEDERAL DE MATO GROSSO
1                      2                           UNIVERSIDADE DE BRASÍLIA
2                      3                    UNIVERSIDADE FEDERAL DE SERGIPE
3                      4                   UNIVERSIDADE FEDERAL DO AMAZONAS
4                      5                      UNIVERSIDADE FEDERAL DO PIAUÍ
...                  ...                                                ...
3874               27076  Escola de Saúde Pública do Estado de Minas Gerais
3875               27103                   UNIVERSIDADE DO DISTRITO FEDERAL
3876               29079      Centro de Estudos e Aperfeiçoamento Funcional
3877               29208  FUNDAÇÃO ESCOLA SUPERIOR DO MINISTÉRIO PÚBLICO...
3878               29224  ESCOLA DA DEFENSORIA PÚBLICA DO ESTADO DE SÃO ...

[3879 rows x 2 columns]


In [ ]:
query = """
SELECT DISTINCT
  CODIGO_IES AS CODIGO_INSTITUICAO, CODIGO_MUNICIPIO
FROM
  graduacao
UNION
SELECT DISTINCT
  CODIGO_IES AS CODIGO_INSTITUICAO, CODIGO_MUNICIPIO
FROM
  especializacao
"""

df = pd.read_sql_query(query, conn)
df.to_sql('Local_Instituicao', conn, if_exists='replace', index=False)

cursor.execute("ALTER TABLE graduacao DROP COLUMN CODIGO_MUNICIPIO;")
cursor.execute("ALTER TABLE especializacao DROP COLUMN CODIGO_MUNICIPIO;")

conn.commit()

print (df)

       CODIGO_INSTITUICAO  CODIGO_MUNICIPIO
0                       1           5100201
1                       1           5100300
2                       1           5101308
3                       1           5101407
4                       1           5101704
...                   ...               ...
37921               27076           3106200
37922               27103           5300108
37923               29079           5103403
37924               29208           5103403
37925               29224           3550308

[37926 rows x 2 columns]


In [ ]:
cursor.execute("ALTER TABLE Graduacao RENAME COLUMN CODIGO_IES TO COD_INSTITUICAO;")

cursor.execute("ALTER TABLE Especializacao RENAME COLUMN CODIGO_IES TO COD_INSTITUICAO;")

cursor.execute("ALTER TABLE Especializacao RENAME COLUMN CODIGO_OCDE_CINE TO COD_AREA_CONHECIMENTO;")

cursor.execute("ALTER TABLE Graduacao RENAME COLUMN CODIGO_AREA_OCDE_CINE TO COD_AREA_CONHECIMENTO;")

cursor.execute("ALTER TABLE Instituicao RENAME COLUMN CODIGO_INSTITUICAO TO COD_INSTITUICAO;")

conn.commit()

## Resultado final

In [ ]:
# Consulta para obter os nomes das tabelas no banco de dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Exibir informações sobre as tabelas
print("Informações das tabelas no banco de dados:")
for table in tables:
    table_name = table[0]
    print(f"\nNome da Tabela: {table_name}")

    # Consulta para obter o número de linhas na tabela
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    rows_count = cursor.fetchone()[0]
    print(f"Número de Linhas: {rows_count}")

    # Consulta para obter os nomes e tipos de dados das colunas da tabela
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    print("Colunas e Tipos de Dados:")
    for column in columns:
        print(f"  Nome da Coluna: {column[1]}, Tipo de Dado: {column[2]}")

Informações das tabelas no banco de dados:

Nome da Tabela: Graduacao
Número de Linhas: 902676
Colunas e Tipos de Dados:
  Nome da Coluna: COD_INSTITUICAO, Tipo de Dado: INTEGER
  Nome da Coluna: CODIGO_CURSO, Tipo de Dado: INTEGER
  Nome da Coluna: NOME_CURSO, Tipo de Dado: TEXT
  Nome da Coluna: GRAU, Tipo de Dado: TEXT
  Nome da Coluna: MODALIDADE, Tipo de Dado: TEXT
  Nome da Coluna: SITUACAO_CURSO, Tipo de Dado: TEXT
  Nome da Coluna: QT_VAGAS_AUTORIZADAS, Tipo de Dado: INTEGER
  Nome da Coluna: CARGA_HORARIA, Tipo de Dado: INTEGER
  Nome da Coluna: COD_AREA_CONHECIMENTO, Tipo de Dado: TEXT

Nome da Tabela: Especializacao
Número de Linhas: 677787
Colunas e Tipos de Dados:
  Nome da Coluna: COD_INSTITUICAO, Tipo de Dado: INTEGER
  Nome da Coluna: COD_DA_ESPECIALIZACAO, Tipo de Dado: INTEGER
  Nome da Coluna: NOME_ESPECIALIZACAO, Tipo de Dado: TEXT
  Nome da Coluna: COD_AREA_CONHECIMENTO, Tipo de Dado: REAL
  Nome da Coluna: CARGA_HORARIA, Tipo de Dado: INTEGER
  Nome da Coluna: DUR

## 6.1 Duas consultas envolvendo seleção e projeção

### 6.1.1 Consulta 1
//selecao de cursos de graduacao a distancia

In [ ]:
query = """
SELECT DISTINCT
  NOME_CURSO
FROM
  GRADUACAO
WHERE
  GRADUACAO.MODALIDADE = 'Educação a Distância'
"""

df = pd.read_sql_query(query, conn)
df

,NOME_CURSO
0,ENGENHARIA AGRONÔMICA
1,LETRAS
2,GEOGRAFIA
3,EDUCAÇÃO FÍSICA
4,SERVIÇOS PENAIS
...,...
630,BUSINESS INTELLIGENCE
631,LETRAS - LITERATURA
632,CIÊNCIA DO CONSUMO
633,LETRAS: PORTUGUÊS LITERATURA


### 6.1.2 Consulta 2
//selecao de cursos de graduacao com carga horaria maior ou igual a 4000 horas

In [ ]:
query = """
SELECT DISTINCT
  NOME_CURSO
FROM
  GRADUACAO
WHERE
  CARGA_HORARIA >= 4000
"""

df = pd.read_sql_query(query, conn)
df

,NOME_CURSO
0,ENGENHARIA AGRONÔMICA
1,EDUCAÇÃO FÍSICA
2,FARMÁCIA
3,LETRAS - LIBRAS - LÍNGUA PORTUGUESA
4,PSICOLOGIA
...,...
389,ENGENHARIA DE INOVAÇÃO
390,QUÍMICA - HABILITAÇÃO TECNOLÓGICA - ÊNFASE AMB...
391,ENGENHARIA DE BIOTECNOLOGIA
392,FARMÁCIA COM HABILITAÇÃO EM BIOQUÍMICO ANALISTA


## 6.2 Três consultas envolvendo junção de duas relações

### 6.2.1 Consulta 3
//instituicoes de ensino que ofertam tanto cursos de graduacao quanto de especializacao


In [ ]:
query = """
SELECT DISTINCT
  NOME_INSTITUICAO
FROM
  GRADUACAO, ESPECIALIZACAO, INSTITUICAO
WHERE
  GRADUACAO.COD_INSTITUICAO = ESPECIALIZACAO.COD_INSTITUICAO
  AND GRADUACAO.COD_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO;
"""

df = pd.read_sql_query(query, conn)
df

### 6.2.2 Consulta 4
//cursos de graduacao ofertados no municipio de Belo Horizonte

In [ ]:
query = """
SELECT DISTINCT
  NOME_CURSO
FROM
  GRADUACAO, INSTITUICAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.NOME_MUNICIPIO = 'Belo Horizonte'

"""

df = pd.read_sql_query(query, conn)
df

### 6.2.3 Consulta 5
//cursos de especializacao ofertados no municipio de Belo Horizonte

In [ ]:
query = """
SELECT DISTINCT
  NOME_ESPECIALIZACAO
FROM
  ESPECIALIZACAO, INSTITUICAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.NOME_MUNICIPIO = 'Belo Horizonte'
"""

df = pd.read_sql_query(query, conn)
df

,NOME_ESPECIALIZACAO
0,Aprendizagem e Neurociência no Contexto Escolar
1,Arquitetura Comercial
2,Audiovisual e Convergência de Mídias
3,Automação Industrial
4,Automação e Indústria 4.0
...,...
52476,Curso de Pós-Graduação em Gestão Estratégica e...
52477,Curso de Pós-Graduação em Gestão e Direito Amb...
52478,Master in data and Decision Science
52479,Product Growth


## 6.3 Três consultas envolvendo junção de três ou mais relações


### 6.3.1 Consulta 6
//instituicoes de ensino localizadas em Belo Horizonte que ofertam tanto cursos de graduacao quanto de especializacao

In [ ]:
query = """
SELECT DISTINCT
  NOME_INSTITUICAO
FROM
  GRADUACAO, ESPECIALIZACAO, INSTITUICAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.NOME_MUNICIPIO = 'Belo Horizonte' AND LOCAL_INSTITUICAO.CODIGO_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO
"""

df = pd.read_sql_query(query, conn)
df

,NOME_INSTITUICAO
0,UNIVERSIDADE DO VALE DO RIO DOS SINOS
1,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO RIO GRANDE...
2,UNIVERSIDADE PRESBITERIANA MACKENZIE
3,Centro Universitário Vale do Rio Verde
4,UNIVERSIDADE JOSÉ DO ROSÁRIO VELLANO
...,...
282,Faculdade FaCiencia
283,Instituto Brasil de Ensino Superior
284,Faculdade Brasil Empreender
285,ACADEMIA DE POLÍCIA MILITAR DE MINAS GERAIS


### 6.3.2 Consulta 7
//municipios que possuem instituicoes de ensino tanto de graduacao quanto de especializacao


In [ ]:
query = """
SELECT DISTINCT
  NOME_MUNICIPIO
FROM
  GRADUACAO, ESPECIALIZACAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
"""

df = pd.read_sql_query(query, conn)
df

### 6.3.3 Consulta 8
//curso de especializacao e de graduacao ofertados no estado de Minas Gerais


In [ ]:
query = """
SELECT DISTINCT
  NOME_CURSO
FROM
  GRADUACAO, ESPECIALIZACAO, LOCAL_INSTITUICAO, MUNICIPIO
WHERE
  (GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO OR ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO) AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.UF = 'MG'
"""

df = pd.read_sql_query(query, conn)
df

## 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

### 6.4.1 Consulta 9
//vagas totais de especializacao e graduacao em Minas Gerais

In [ ]:
query = """
SELECT DISTINCT
  QT_VAGAS_AUTORIZADAS
FROM
  GRADUACAO, ESPECIALIZACAO, LOCAL_INSTITUICAO
WHERE
  (GRADUACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO OR ESPECIALIZACAO.COD_INSTITUICAO = LOCAL_INSTITUICAO.CODIGO_INSTITUICAO) AND LOCAL_INSTITUICAO.CODIGO_MUNICIPIO = MUNICIPIO.CODIGO_MUNICIPIO
  AND MUNICIPIO.UF = 'MG'
GROUP BY
  GRADUACAO.QT_VAGAS_AUTORIZADAS, ESPECIALIZACAO.QT_VAGAS_AUTORIZADAS
"""

df = pd.read_sql_query(query, conn)
df

### 6.4.2 Consulta 10
//instituicoes ordenadas por numero de vagas de graduacao e especializacao

In [ ]:
query = """
SELECT DISTINCT
  NOME_INSTITUICAO
FROM
  GRADUACAO, ESPECIALIZACAO, INSTITUICAO
WHERE
  GRADUACAO.COD_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO OR ESPECIALIZACAO.COD_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO
ORDER BY
  GRADUACAO.QT_VAGAS_AUTORIZADAS, ESPECIALIZACAO.QT_VAGAS_AUTORIZADAS DESC
"""

df = pd.read_sql_query(query, conn)
df

# 7. Autoavaliação dos membros

## Chrystian Melo 10/10
## Fernando Coutinho  10/10
## Murillo Ribeiro  10/10
## Nauan Barbosa 10/10

In [ ]:
query = """
SELECT DISTINCT

FROM
  Tematica, ESPECIALIZACAO, INSTITUICAO
WHERE
  GRADUACAO.COD_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO OR ESPECIALIZACAO.COD_INSTITUICAO = INSTITUICAO.COD_INSTITUICAO
ORDER BY
  GRADUACAO.QT_VAGAS_AUTORIZADAS, ESPECIALIZACAO.QT_VAGAS_AUTORIZADAS DESC
"""

df = pd.read_sql_query(query, conn)
df